# Hill Climbing Method

In [1]:
def hillClimbingOptimization(optProblem, stopCount, numRestarts, simulatedAnnealing):
    finalBest = optProblem.getCurrentInput()
    
    restartNum = 1
    restartData = []
    
    bestIterationData = []

    
    for i in range(1, numRestarts + 1):
        if not simulatedAnnealing:
            optProblem.reInit()
        counter = 0
        
        iterationNum = 1
        iterationData = []
        
        while counter < stopCount:
            print("current accuracy is %f" % optProblem.evaluate(optProblem.getCurrentInput()))
            
            potentialX = optProblem.getRandomSuccessor()
            if optProblem.evaluate(optProblem.getCurrentInput()) < optProblem.evaluate(potentialX):  
                counter = 0
                optProblem.setCurrentInput(potentialX)
            else:
                counter += 1
                
            #second graph
            iterationData.append((iterationNum, optProblem.evaluate(optProblem.getCurrentInput())))
            iterationNum += 1
            
        bestX = optProblem.getCurrentInput()
        
        #Graph of Restarts vs Accuracy
        restartData.append((restartNum, optProblem.evaluate(bestX)))
        restartNum += 1
        
        print(iterationData)
        if restartNum == 2 or optProblem.evaluate(bestX) > optProblem.evaluate(finalBest):
            finalBest = bestX
            bestIterationData = iterationData
        
    return (restartData, bestIterationData)

# Simulated Annealing

In [2]:
import numpy as np
def simulatedAnnealingAlgorithm(optProblem, tempDecrease, stopTemp, initTemperature):    
    temperature = initTemperature
    
    iterationNum = 1
    iterationData = []
    while temperature > stopTemp:
        print("current accuracy is %f" % optProblem.evaluate(optProblem.getCurrentInput()))
        
        potentialX = optProblem.getRandomSuccessor()
        
#         print(optProblem.getCurrentInput())
#         print(potentialX)
        
        currentY = optProblem.evaluate(optProblem.getCurrentInput())
        potentialY = optProblem.evaluate(potentialX)

        
        if currentY < potentialY:
            optProblem.setCurrentInput(potentialX)
            print("better input")
        else:
            #check if need to go to new input even it the input is worse based on equation
            print("probability of accepting lower is %f" % np.exp((potentialY - currentY)/temperature))
            if random.random() < np.exp((potentialY - currentY)/temperature):
                optProblem.setCurrentInput(potentialX)
    
        temperature *= tempDecrease
        iterationData.append((iterationNum, optProblem.evaluate(optProblem.getCurrentInput())))
        iterationNum += 1
        
    bestX = optProblem.getCurrentInput()
    return iterationData       
    

# Genetic Algorithm

In [3]:
def geneticAlgorithm(optProblem, probMutate, numIterations):
    iterationNum = 1
    iterationData = []
    for i in range(0, numIterations + 1):
        bestAccuracy = optProblem.crossoverAndSelect(probMutate)
        iterationData.append((iterationNum, bestAccuracy))
        iterationNum += 1
    return iterationData
        

# Neural Net

In [4]:
import random
import numpy as np
import itertools
import pandas as pd
import keras
import operator

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.ensemble import AdaBoostClassifier
import random

def initializeInputs(numSpecies, numFeatures, numClasses):
    inputs = []
    model = setupModel(numFeatures, numClasses)
    inputs.append(model.get_weights())
    for i in range (1, numSpecies):
        inputs.append(getRandomInitialSuccessor(200, .1, inputs[i-1]))
    return inputs

def getRandomInitialSuccessor(neighborhood, alpha, currentInput):
        randomNums = list(np.arange(-1*neighborhood, neighborhood, alpha))
        successor = [np.copy(arr) for arr in currentInput]
        for i in range(0,len(successor), 2):
            array = successor[i]
            for x,y in itertools.product(list(range(0,len(array))), list(range(0,len(array[0])))):
                array[x,y] += random.sample(randomNums, 1)
        return successor

class NeuralNetProblem():
    def __init__(self, X_train, y_train, X_test, y_test, numFeatures, numClasses, neighborhood, alpha, numSpecies):
        #store the data
        self.X_train = X_train
        self.y_train = y_train
        self.X_test = X_test
        self.y_test = y_test
        #setup the model
        self.numFeatures = numFeatures
        self.numClasses = numClasses
        self.model = setupModel(numFeatures, numClasses)
        #get the initial weights
        self.neighborhood = neighborhood
        self.alpha = alpha
        self.currentInput = self.model.get_weights()
        self.numSpecies = numSpecies
        self.species = initializeInputs(numSpecies, numFeatures, numClasses)
    
    def reInit(self):
        self.model = setupModel(self.numFeatures, self.numClasses)
        self.currentInput = self.model.get_weights()
        
    def getCurrentInput(self):
        return self.currentInput
    
    def setCurrentInput(self, weights):
        self.currentInput = weights
        self.model.set_weights(weights)
    
    def evaluate(self, weights):
        self.model.set_weights(weights)
        dummy1, dummy2, accuracy = self.model.evaluate(X_train, y_train, batch_size = 32)
        self.model.set_weights(self.currentInput)
        return accuracy
    
    def getRandomSuccessor(self):
        randomNums = list(np.arange(-1*self.neighborhood, self.neighborhood, self.alpha))
        successor = [np.copy(arr) for arr in self.currentInput]
        for i in range(0,len(successor), 2):
            array = successor[i]
            for x,y in itertools.product(list(range(0,len(array))), list(range(0,len(array[0])))):
                array[x,y] += random.sample(randomNums, 1)
        return successor
    
    def mutate(self, prob):
        for i in range(self.numSpecies, len(self.species)):
            if random.random() < prob:
                self.species[i] = self.mutateSpecific(prob, self.species[i])
        
        
    def mutateSpecific(self, prob, species):
        randomNums = list(np.arange(-3*self.neighborhood, 3*self.neighborhood, self.alpha))
        newSpecies = [np.copy(arr) for arr in species]
        
        for i in range(0,len(newSpecies), 2):
            array = newSpecies[i]
            for x,y in itertools.product(list(range(0,len(array))), list(range(0,len(array[0])))):
                if random.random() < prob:
                    array[x,y] += random.sample(randomNums, 1)  
        return newSpecies
        
    def crossoverAndSelect(self, prob):
        for x,y in random.sample(list(itertools.product(list(range(0,self.numSpecies)), list(range(0,self.numSpecies)))), self.numSpecies*2):
            if x is not y:
                self.species.append(self.breed(self.species[x], self.species[y]))
        #mutate the species
        self.mutate(prob)
        #choose the best species
        intermediate = [(i, self.evaluate(i)) for i in self.species]
        intermediate.sort(key = operator.itemgetter(1), reverse = True)
        #print the best accuracies
        for i in range(0, self.numSpecies):
            print("accuracy of organism %d" % i)
            print(intermediate[i][1])
        bestAccuracy = intermediate[0][1]
        self.species = [intermediate[i][0] for i in range(0, self.numSpecies)]
        return bestAccuracy
        
        
    
    def breed(self, species1, species2):
        newSpecies = [np.copy(arr) for arr in species1]
#         print("newSpecies")
#         print(newSpecies)
#         print("species1")
#         print(species1)
#         print
        for i in range(0,len(newSpecies), 2):
            array = newSpecies[i]
            arrayS1 = species1[i]
            arrayS2 = species2[i]
            for x,y in itertools.product(list(range(0,len(array))), list(range(0,len(array[0])))):
                array[x,y] = (arrayS1[x,y] + arrayS2[x,y])/2.0
        return newSpecies
        
    def getSpecies(self):
        return self.species
        
        

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [5]:
def importData(fileName):
    finalData = pd.read_csv(fileName)
    finalData.count()

    from sklearn import preprocessing
    

    for column in finalData.columns:
        if finalData[column].dtype == type(object):
            le = preprocessing.LabelEncoder()
            finalData[column] = le.fit_transform(finalData[column])
            
    return finalData.sample(frac = 1)

In [6]:
def setupDataNN(finalData, fraction, numClasses):
    
    train, test = train_test_split(finalData, test_size = 0.2)
    train = train.sample(frac = fraction)
    
    features = ['Parents', 'has_nurs', 'form', 'children', 'housing', 'finance', 'social', 'health']
    
    X_train = train[features]
    y_train = train['app_status']

    X_test = test[features]
    y_test = test["app_status"]
    
    
    y_train = keras.utils.to_categorical(y_train, num_classes = numClasses)
    y_test = keras.utils.to_categorical(y_test, num_classes = numClasses)
    
    return (X_train, y_train, X_test, y_test)

In [7]:
def setupModel(numFeatures, numClasses):
    import keras
    from keras.models import Sequential
    from keras.layers import Dense
    from keras.layers import Dropout
    from keras import optimizers
    import random
    
    model = Sequential()
    model.add(Dense(55, input_dim = numFeatures, activation = 'sigmoid'))
    model.add(Dense(55, activation = 'sigmoid'))
    model.add(Dense(numClasses, activation = 'sigmoid'))
    model.summary()
    
    model.compile(loss = 'mean_squared_error', optimizer = "adam", metrics=["mean_squared_error", 'accuracy'])

    
    return model

In [8]:
data = importData("finalDataNursing.csv")
X_train, y_train, X_test, y_test = setupDataNN(data, 1, 3)
optProblem = NeuralNetProblem(X_train, y_train, X_test, y_test, 8, 3, 1.5, .3, 0)

restartData, bestIterationData = hillClimbingOptimization(optProblem, 100, 5, False)
print(bestIterationData)

#store the restart data
numRestarts, fitness = [i[0] for i in restartData], [i[1] for i in restartData]
plotData = [('Restart Number', numRestarts), ('Fitness', fitness)]
df = pd.DataFrame.from_items(plotData)
df.to_csv("hillClimbingNNRestarts.csv")

#store the iteration data
numRestarts, fitness = [i[0] for i in bestIterationData], [i[1] for i in bestIterationData]
plotData = [('Restart Number', numRestarts), ('Fitness', fitness)]
df = pd.DataFrame.from_items(plotData)
df.to_csv("hillClimbingIterationsNN.csv")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 55)                495       
_________________________________________________________________
dense_2 (Dense)              (None, 55)                3080      
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 168       
Total params: 3,743
Trainable params: 3,743
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 55)                495       
_________________________________________________________________
dense_5 (Dense)              (None, 55)                3080      
_________________________________________________________________
dense_6 

10104/10104 [==============================] - 0s 12us/step
current accuracy is 0.444378
10104/10104 [==============================] - 0s 12us/step
current accuracy is 0.444378
10104/10104 [==============================] - 0s 12us/step
current accuracy is 0.444378
10104/10104 [==============================] - 0s 12us/step
current accuracy is 0.444378
10104/10104 [==============================] - 0s 12us/step
current accuracy is 0.444378
10104/10104 [==============================] - 0s 12us/step
current accuracy is 0.448139
10104/10104 [==============================] - 0s 12us/step
current accuracy is 0.457641
10104/10104 [==============================] - 0s 13us/step
current accuracy is 0.457641
10104/10104 [==============================] - 0s 12us/step
current accuracy is 0.457641
10104/10104 [==============================] - 0s 13us/step
current accuracy is 0.457641
10104/10104 [==============================] - 0s 13us/step
current accuracy is 0.457641
10104/10104 [========

10104/10104 [==============================] - 0s 12us/step
current accuracy is 0.505740
10104/10104 [==============================] - 0s 12us/step
current accuracy is 0.505740
10104/10104 [==============================] - 0s 13us/step
current accuracy is 0.505740
10104/10104 [==============================] - 0s 13us/step
current accuracy is 0.505740
10104/10104 [==============================] - 0s 12us/step
current accuracy is 0.505740
10104/10104 [==============================] - 0s 13us/step
current accuracy is 0.505740
10104/10104 [==============================] - 0s 13us/step
current accuracy is 0.505740
10104/10104 [==============================] - 0s 13us/step
current accuracy is 0.505740
10104/10104 [==============================] - 0s 13us/step
current accuracy is 0.505740
10104/10104 [==============================] - 0s 13us/step
current accuracy is 0.505740
10104/10104 [==============================] - 0s 12us/step
current accuracy is 0.505740
10104/10104 [========

10104/10104 [==============================] - 0s 12us/step
current accuracy is 0.507918
10104/10104 [==============================] - 0s 12us/step
current accuracy is 0.507918
10104/10104 [==============================] - 0s 12us/step
current accuracy is 0.507918
10104/10104 [==============================] - 0s 13us/step
current accuracy is 0.507918
10104/10104 [==============================] - 0s 12us/step
current accuracy is 0.507918
10104/10104 [==============================] - 0s 12us/step
current accuracy is 0.507918
10104/10104 [==============================] - 0s 13us/step
current accuracy is 0.507918
10104/10104 [==============================] - 0s 12us/step
current accuracy is 0.507918
10104/10104 [==============================] - 0s 13us/step
current accuracy is 0.507918
10104/10104 [==============================] - 0s 11us/step
current accuracy is 0.507918
10104/10104 [==============================] - 0s 13us/step
current accuracy is 0.507918
10104/10104 [========

10104/10104 [==============================] - 0s 12us/step
current accuracy is 0.523357
10104/10104 [==============================] - 0s 12us/step
current accuracy is 0.523357
10104/10104 [==============================] - 0s 12us/step
current accuracy is 0.523357
10104/10104 [==============================] - 0s 12us/step
current accuracy is 0.523753
10104/10104 [==============================] - 0s 12us/step
current accuracy is 0.523753
10104/10104 [==============================] - 0s 12us/step
current accuracy is 0.523753
10104/10104 [==============================] - 0s 12us/step
current accuracy is 0.523753
10104/10104 [==============================] - 0s 12us/step
current accuracy is 0.523753
10104/10104 [==============================] - 0s 12us/step
current accuracy is 0.523753
10104/10104 [==============================] - 0s 12us/step
current accuracy is 0.523753
10104/10104 [==============================] - 0s 15us/step
current accuracy is 0.523753
10104/10104 [========

...

10104/10104 [==============================] - 0s 12us/step
current accuracy is 0.555721
10104/10104 [==============================] - 0s 12us/step
current accuracy is 0.555721
10104/10104 [==============================] - 0s 13us/step
current accuracy is 0.555721
10104/10104 [==============================] - 0s 13us/step
current accuracy is 0.555721
10104/10104 [==============================] - 0s 11us/step
current accuracy is 0.555721
10104/10104 [==============================] - 0s 11us/step
current accuracy is 0.555721
10104/10104 [==============================] - 0s 14us/step
current accuracy is 0.555721
10104/10104 [==============================] - 0s 11us/step
current accuracy is 0.555721
10104/10104 [==============================] - 0s 12us/step
current accuracy is 0.555721
10104/10104 [==============================] - 0s 11us/step
current accuracy is 0.555721
10104/10104 [==============================] - 0s 11us/step
current accuracy is 0.555721
10104/10104 [========

10104/10104 [==============================] - 0s 12us/step
current accuracy is 0.608670
10104/10104 [==============================] - 0s 12us/step
current accuracy is 0.623120
10104/10104 [==============================] - 0s 12us/step
current accuracy is 0.623120
10104/10104 [==============================] - 0s 12us/step
current accuracy is 0.627870
10104/10104 [==============================] - 0s 13us/step
current accuracy is 0.627870
10104/10104 [==============================] - 0s 11us/step
current accuracy is 0.627870
10104/10104 [==============================] - 0s 12us/step
current accuracy is 0.627870
10104/10104 [==============================] - 0s 13us/step
current accuracy is 0.627870
10104/10104 [==============================] - 0s 14us/step
current accuracy is 0.627870
10104/10104 [==============================] - 0s 13us/step
current accuracy is 0.627870
10104/10104 [==============================] - 0s 13us/step
current accuracy is 0.627870
10104/10104 [========

10104/10104 [==============================] - 0s 12us/step
current accuracy is 0.658650
10104/10104 [==============================] - 0s 12us/step
current accuracy is 0.658650
10104/10104 [==============================] - 0s 12us/step
current accuracy is 0.658650
10104/10104 [==============================] - 0s 13us/step
current accuracy is 0.658650
10104/10104 [==============================] - 0s 11us/step
current accuracy is 0.658650
10104/10104 [==============================] - 0s 12us/step
current accuracy is 0.658650
10104/10104 [==============================] - 0s 12us/step
current accuracy is 0.658650
10104/10104 [==============================] - 0s 13us/step
current accuracy is 0.658650
10104/10104 [==============================] - 0s 10us/step
current accuracy is 0.658650
10104/10104 [==============================] - 0s 12us/step
current accuracy is 0.658650
10104/10104 [==============================] - 0s 12us/step
current accuracy is 0.658650
10104/10104 [========

10104/10104 [==============================] - 0s 14us/step
current accuracy is 0.658650
10104/10104 [==============================] - 0s 13us/step
current accuracy is 0.658650
10104/10104 [==============================] - 0s 13us/step
current accuracy is 0.658650
10104/10104 [==============================] - 0s 14us/step
current accuracy is 0.658650
10104/10104 [==============================] - 0s 16us/step
current accuracy is 0.658650
10104/10104 [==============================] - 0s 13us/step
current accuracy is 0.658650
10104/10104 [==============================] - 0s 17us/step
current accuracy is 0.658650
10104/10104 [==============================] - 0s 15us/step
current accuracy is 0.658650
10104/10104 [==============================] - 0s 13us/step
current accuracy is 0.658650
10104/10104 [==============================] - 0s 13us/step
current accuracy is 0.658650
10104/10104 [==============================] - 0s 13us/step
current accuracy is 0.658650
10104/10104 [========

KeyboardInterrupt: 

In [9]:
data = importData("finalDataNursing.csv")
X_train, y_train, X_test, y_test = setupDataNN(data, 1, 3)
optProblem = NeuralNetProblem(X_train, y_train, X_test, y_test, 8, 3, 6.5, .3, 0)

bestIterationData = simulatedAnnealingAlgorithm(optProblem, .90, .000000000000000002, 3)
#bestInput, accuracy, counter = hillClimbingOptimization(optProblem, 100, 1, True)

#store the iteration data
numRestarts, fitness = [i[0] for i in bestIterationData], [i[1] for i in bestIterationData]
plotData = [('Iteration', numRestarts), ('Fitness', fitness)]
df = pd.DataFrame.from_items(plotData)
df.to_csv("simulatedAnnealingIterationsNN.csv")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_22 (Dense)             (None, 55)                495       
_________________________________________________________________
dense_23 (Dense)             (None, 55)                3080      
_________________________________________________________________
dense_24 (Dense)             (None, 3)                 168       
Total params: 3,743
Trainable params: 3,743
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_25 (Dense)             (None, 55)                495       
_________________________________________________________________
dense_26 (Dense)             (None, 55)                3080      
_________________________________________________________________
dense_27

10104/10104 [==============================] - 0s 18us/step
current accuracy is 0.374802
10104/10104 [==============================] - 0s 19us/step
better input
10104/10104 [==============================] - 0s 18us/step
current accuracy is 0.383215
10104/10104 [==============================] - 0s 18us/step
probability of accepting lower is 0.867406
10104/10104 [==============================] - 0s 18us/step
current accuracy is 0.381235
10104/10104 [==============================] - 0s 18us/step
probability of accepting lower is 0.215861
10104/10104 [==============================] - 0s 19us/step
current accuracy is 0.362035
10104/10104 [==============================] - 0s 18us/step
better input
10104/10104 [==============================] - 0s 20us/step
current accuracy is 0.367181
10104/10104 [==============================] - 0s 18us/step
better input
10104/10104 [==============================] - 0s 18us/step
current accuracy is 0.375297
10104/10104 [============================

10104/10104 [==============================] - 0s 24us/step
probability of accepting lower is 0.000000
10104/10104 [==============================] - 0s 24us/step
current accuracy is 0.448931
10104/10104 [==============================] - 0s 20us/step
probability of accepting lower is 0.000000
10104/10104 [==============================] - 0s 18us/step
current accuracy is 0.448931
10104/10104 [==============================] - 0s 18us/step
probability of accepting lower is 0.000000
10104/10104 [==============================] - 0s 18us/step
current accuracy is 0.448931
10104/10104 [==============================] - 0s 18us/step
probability of accepting lower is 0.000000
10104/10104 [==============================] - 0s 18us/step
current accuracy is 0.448931
10104/10104 [==============================] - 0s 22us/step
probability of accepting lower is 0.000000
10104/10104 [==============================] - 0s 18us/step
current accuracy is 0.448931
10104/10104 [===========================

10104/10104 [==============================] - 0s 18us/step
probability of accepting lower is 0.000000
10104/10104 [==============================] - 0s 18us/step
current accuracy is 0.498713
10104/10104 [==============================] - 0s 19us/step
probability of accepting lower is 0.000000
10104/10104 [==============================] - 0s 18us/step
current accuracy is 0.498713
10104/10104 [==============================] - 0s 18us/step
probability of accepting lower is 0.000000
10104/10104 [==============================] - 0s 18us/step
current accuracy is 0.498713
10104/10104 [==============================] - 0s 18us/step
probability of accepting lower is 0.000000
10104/10104 [==============================] - 0s 23us/step
current accuracy is 0.498713
10104/10104 [==============================] - 0s 18us/step
probability of accepting lower is 0.000000
10104/10104 [==============================] - 0s 17us/step
current accuracy is 0.498713
10104/10104 [===========================

10104/10104 [==============================] - 0s 19us/step
probability of accepting lower is 0.000000
10104/10104 [==============================] - 0s 18us/step
current accuracy is 0.530681
10104/10104 [==============================] - 0s 17us/step
probability of accepting lower is 0.000000
10104/10104 [==============================] - 0s 18us/step
current accuracy is 0.530681
10104/10104 [==============================] - 0s 18us/step
better input
10104/10104 [==============================] - 0s 18us/step
current accuracy is 0.538994
10104/10104 [==============================] - 0s 18us/step
probability of accepting lower is 0.000000
10104/10104 [==============================] - 0s 18us/step
current accuracy is 0.538994
10104/10104 [==============================] - 0s 18us/step
probability of accepting lower is 0.000000
10104/10104 [==============================] - 0s 18us/step
current accuracy is 0.538994
10104/10104 [==============================] - 0s 25us/step
probabilit

10104/10104 [==============================] - 0s 20us/step
probability of accepting lower is 0.000000
10104/10104 [==============================] - 0s 21us/step
current accuracy is 0.567597
10104/10104 [==============================] - 0s 26us/step
probability of accepting lower is 0.000000
10104/10104 [==============================] - 0s 18us/step
current accuracy is 0.567597
10104/10104 [==============================] - 0s 19us/step
probability of accepting lower is 0.000000
10104/10104 [==============================] - 0s 18us/step
current accuracy is 0.567597
10104/10104 [==============================] - 0s 20us/step
probability of accepting lower is 0.000000
10104/10104 [==============================] - 0s 19us/step
current accuracy is 0.567597
10104/10104 [==============================] - 0s 19us/step
probability of accepting lower is 0.000000
10104/10104 [==============================] - 0s 19us/step
current accuracy is 0.567597
10104/10104 [===========================

10104/10104 [==============================] - 0s 18us/step
current accuracy is 0.581057
10104/10104 [==============================] - 0s 18us/step
probability of accepting lower is 0.000000
10104/10104 [==============================] - 0s 18us/step
current accuracy is 0.581057
10104/10104 [==============================] - 0s 18us/step
probability of accepting lower is 0.000000
10104/10104 [==============================] - 0s 18us/step
current accuracy is 0.581057
10104/10104 [==============================] - 0s 18us/step
probability of accepting lower is 0.000000
10104/10104 [==============================] - 0s 20us/step
current accuracy is 0.581057
10104/10104 [==============================] - 0s 18us/step
probability of accepting lower is 0.000000
10104/10104 [==============================] - 0s 19us/step
current accuracy is 0.581057
10104/10104 [==============================] - 0s 19us/step
probability of accepting lower is 0.000000
10104/10104 [===========================

10104/10104 [==============================] - 0s 19us/step
probability of accepting lower is 0.000000
10104/10104 [==============================] - 0s 32us/step
current accuracy is 0.592142
10104/10104 [==============================] - 0s 18us/step
probability of accepting lower is 0.000000
10104/10104 [==============================] - 0s 18us/step
current accuracy is 0.592142
10104/10104 [==============================] - 0s 20us/step
probability of accepting lower is 0.000000
10104/10104 [==============================] - 0s 27us/step
current accuracy is 0.592142
10104/10104 [==============================] - 0s 25us/step
probability of accepting lower is 0.000000
10104/10104 [==============================] - 0s 24us/step
current accuracy is 0.592142
10104/10104 [==============================] - 0s 19us/step
probability of accepting lower is 0.000000
10104/10104 [==============================] - 0s 20us/step
current accuracy is 0.592142
10104/10104 [===========================

In [10]:
data = importData("finalDataNursing.csv")
X_train, y_train, X_test, y_test = setupDataNN(data, 1, 3)
optProblem = NeuralNetProblem(X_train, y_train, X_test, y_test, 8, 3, 10.5, .3, 40)

bestIterationData = geneticAlgorithm(optProblem, 0.8, 300)

#store the iteration data
numRestarts, fitness = [i[0] for i in bestIterationData], [i[1] for i in bestIterationData]
plotData = [('Iteration', numRestarts), ('Fitness of best Organism', fitness)]
df = pd.DataFrame.from_items(plotData)
df.to_csv("geneticAlgorithmIterationsNN.csv")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_28 (Dense)             (None, 55)                495       
_________________________________________________________________
dense_29 (Dense)             (None, 55)                3080      
_________________________________________________________________
dense_30 (Dense)             (None, 3)                 168       
Total params: 3,743
Trainable params: 3,743
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_31 (Dense)             (None, 55)                495       
_________________________________________________________________
dense_32 (Dense)             (None, 55)                3080      
_________________________________________________________________
dense_33

10104/10104 [==============================] - 0s 18us/step
accuracy of organism 0
0.3671813143545545
accuracy of organism 1
0.3640142517578762
accuracy of organism 2
0.3497624703087886
accuracy of organism 3
0.34471496437054633
accuracy of organism 4
0.33917260490894696
accuracy of organism 5
0.33917260490894696
accuracy of organism 6
0.3387767220902613
accuracy of organism 7
0.338578780657322
accuracy of organism 8
0.3382818685669042
accuracy of organism 9
0.3382818685669042
accuracy of organism 10
0.33818289786223277
accuracy of organism 11
0.33818289786223277
accuracy of organism 12
0.33818289786223277
accuracy of organism 13
0.33818289786223277
accuracy of organism 14
0.33818289786223277
accuracy of organism 15
0.33818289786223277
accuracy of organism 16
0.33818289786223277
accuracy of organism 17
0.33808392715756136
accuracy of organism 18
0.33808392715756136
accuracy of organism 19
0.33808392715756136
accuracy of organism 20
0.33808392715756136
accuracy of organism 21
0.33808392

10104/10104 [==============================] - 0s 18us/step
accuracy of organism 0
0.3745051464766429
accuracy of organism 1
0.3671813143545545
accuracy of organism 2
0.3640142517578762
accuracy of organism 3
0.3614410134364194
accuracy of organism 4
0.3514449722882027
accuracy of organism 5
0.35124703087885983
accuracy of organism 6
0.351049089469517
accuracy of organism 7
0.3497624703087886
accuracy of organism 8
0.34709026128266035
accuracy of organism 9
0.34501187648456055
accuracy of organism 10
0.3449129057562927
accuracy of organism 11
0.34471496437054633
accuracy of organism 12
0.34441805225653205
accuracy of organism 13
0.3439231987567714
accuracy of organism 14
0.34382422802850354
accuracy of organism 15
0.34224069677735736
accuracy of organism 16
0.3394695170229612
accuracy of organism 17
0.3393705463182898
accuracy of organism 18
0.33927157561361837
accuracy of organism 19
0.33927157561361837
accuracy of organism 20
0.33917260490894696
accuracy of organism 21
0.339172604908

10104/10104 [==============================] - 0s 19us/step
accuracy of organism 0
0.3745051464766429
accuracy of organism 1
0.3745051464766429
accuracy of organism 2
0.37074425969912905
accuracy of organism 3
0.3700514647428327
accuracy of organism 4
0.36846793349168644
accuracy of organism 5
0.3683689628106115
accuracy of organism 6
0.3673792557403009
accuracy of organism 7
0.3671813143545545
accuracy of organism 8
0.3653998416940654
accuracy of organism 9
0.3649049881471119
accuracy of organism 10
0.3645091052812333
accuracy of organism 11
0.3640142517578762
accuracy of organism 12
0.36332145684877276
accuracy of organism 13
0.3626286619396693
accuracy of organism 14
0.3620348377116408
accuracy of organism 15
0.3614410134364194
accuracy of organism 16
0.36104513064133015
accuracy of organism 17
0.36074821850371946
accuracy of organism 18
0.3606492478226445
accuracy of organism 19
0.3605502771179731
accuracy of organism 20
0.3587688044338876
accuracy of organism 21
0.3579770387729197

10104/10104 [==============================] - 0s 18us/step
accuracy of organism 0
0.38370942198748825
accuracy of organism 1
0.38311559775945975
accuracy of organism 2
0.38291765635011693
accuracy of organism 3
0.3807403008709422
accuracy of organism 4
0.3800475059146458
accuracy of organism 5
0.3793547110291388
accuracy of organism 6
0.3785629453445746
accuracy of organism 7
0.37806809184481394
accuracy of organism 8
0.3778701504590676
accuracy of organism 9
0.3777711797543961
accuracy of organism 10
0.37707838479809974
accuracy of organism 11
0.376187648456057
accuracy of organism 12
0.3759897070467142
accuracy of organism 13
0.3757917656609678
accuracy of organism 14
0.3754948535233571
accuracy of organism 15
0.37539588284228215
accuracy of organism 16
0.375
accuracy of organism 17
0.375
accuracy of organism 18
0.3745051464766429
accuracy of organism 19
0.3745051464766429
accuracy of organism 20
0.37440617579556795
accuracy of organism 21
0.37391132222501794
accuracy of organism 22

...

10104/10104 [==============================] - 0s 14us/step
accuracy of organism 0
0.6523159144421182
accuracy of organism 1
0.6523159144421182
accuracy of organism 2
0.6523159144421182
accuracy of organism 3
0.6523159144421182
accuracy of organism 4
0.6523159144421182
accuracy of organism 5
0.6523159144421182
accuracy of organism 6
0.6523159144421182
accuracy of organism 7
0.6523159144421182
accuracy of organism 8
0.6523159144421182
accuracy of organism 9
0.6523159144421182
accuracy of organism 10
0.6523159144421182
accuracy of organism 11
0.6523159144421182
accuracy of organism 12
0.6523159144421182
accuracy of organism 13
0.6523159144421182
accuracy of organism 14
0.6523159144421182
accuracy of organism 15
0.6523159144421182
accuracy of organism 16
0.6523159144421182
accuracy of organism 17
0.6523159144421182
accuracy of organism 18
0.6523159144421182
accuracy of organism 19
0.6523159144421182
accuracy of organism 20
0.6523159144421182
accuracy of organism 21
0.6523159144421182
accu

10104/10104 [==============================] - 0s 19us/step
accuracy of organism 0
0.6523159144421182
accuracy of organism 1
0.6523159144421182
accuracy of organism 2
0.6523159144421182
accuracy of organism 3
0.6523159144421182
accuracy of organism 4
0.6523159144421182
accuracy of organism 5
0.6523159144421182
accuracy of organism 6
0.6523159144421182
accuracy of organism 7
0.6523159144421182
accuracy of organism 8
0.6523159144421182
accuracy of organism 9
0.6523159144421182
accuracy of organism 10
0.6523159144421182
accuracy of organism 11
0.6523159144421182
accuracy of organism 12
0.6523159144421182
accuracy of organism 13
0.6523159144421182
accuracy of organism 14
0.6523159144421182
accuracy of organism 15
0.6523159144421182
accuracy of organism 16
0.6523159144421182
accuracy of organism 17
0.6523159144421182
accuracy of organism 18
0.6523159144421182
accuracy of organism 19
0.6523159144421182
accuracy of organism 20
0.6523159144421182
accuracy of organism 21
0.6523159144421182
accu

accuracy of organism 34
0.6523159144421182
accuracy of organism 35
0.6523159144421182
accuracy of organism 36
0.6523159144421182
accuracy of organism 37
0.6523159144421182
accuracy of organism 38
0.6523159144421182
accuracy of organism 39
0.6523159144421182
10104/10104 [==============================] - 0s 14us/step
accuracy of organism 0
0.6523159144421182
accuracy of organism 1
0.6523159144421182
accuracy of organism 2
0.6523159144421182
accuracy of organism 3
0.6523159144421182
accuracy of organism 4
0.6523159144421182
accuracy of organism 5
0.6523159144421182
accuracy of organism 6
0.6523159144421182
accuracy of organism 7
0.6523159144421182
accuracy of organism 8
0.6523159144421182
accuracy of organism 9
0.6523159144421182
accuracy of organism 10
0.6523159144421182
accuracy of organism 11
0.6523159144421182
accuracy of organism 12
0.6523159144421182
accuracy of organism 13
0.6523159144421182
accuracy of organism 14
0.6523159144421182
accuracy of organism 15
0.6523159144421182
accu

10104/10104 [==============================] - 0s 19us/step
accuracy of organism 0
0.6523159144421182
accuracy of organism 1
0.6523159144421182
accuracy of organism 2
0.6523159144421182
accuracy of organism 3
0.6523159144421182
accuracy of organism 4
0.6523159144421182
accuracy of organism 5
0.6523159144421182
accuracy of organism 6
0.6523159144421182
accuracy of organism 7
0.6523159144421182
accuracy of organism 8
0.6523159144421182
accuracy of organism 9
0.6523159144421182
accuracy of organism 10
0.6523159144421182
accuracy of organism 11
0.6523159144421182
accuracy of organism 12
0.6523159144421182
accuracy of organism 13
0.6523159144421182
accuracy of organism 14
0.6523159144421182
accuracy of organism 15
0.6523159144421182
accuracy of organism 16
0.6523159144421182
accuracy of organism 17
0.6523159144421182
accuracy of organism 18
0.6523159144421182
accuracy of organism 19
0.6523159144421182
accuracy of organism 20
0.6523159144421182
accuracy of organism 21
0.6523159144421182
accu

0.6523159144421182
